In [1]:
%cd '/home/s3/yikyungkim/research/cbr/generator_sep'
%pwd

/home/s3/yikyungkim/research/cbr/generator_sep


'/home/s3/yikyungkim/research/cbr/generator_sep'

In [2]:
from tqdm import tqdm
import json
import os
from datetime import datetime
import time
import logging
from utils import *
from config import parameters as conf
from torch import nn
import torch
import torch.optim as optim

In [3]:
from transformers import RobertaTokenizer
from transformers import RobertaConfig
tokenizer = RobertaTokenizer.from_pretrained(conf.model_size)
model_config = RobertaConfig.from_pretrained(conf.model_size)

In [4]:
if conf.mode == "train":
    # model_dir_name = conf.model_save_name + "_" + \
    #     datetime.now().strftime("%Y%m%d%H%M%S")         # for restart
    model_dir_name = conf.model_save_name
    model_dir = os.path.join(conf.output_path, model_dir_name)
    results_path = os.path.join(model_dir, "results")
    saved_model_path = os.path.join(model_dir, "saved_model")
    os.makedirs(saved_model_path, exist_ok=True)       # for restart 
    os.makedirs(results_path, exist_ok=True)
    log_file = os.path.join(results_path, 'log.txt')

In [5]:
op_list = read_txt(conf.op_list_file, log_file)
op_list = [op + '(' for op in op_list]
op_list = ['EOF', 'UNK', 'GO', ')'] + op_list
const_list = read_txt(conf.const_list_file, log_file)
const_list = [const.lower().replace('.', '_') for const in const_list]
reserved_token_size = len(op_list) + len(const_list)

Reading: /shared/s3/lab07/yikyung/cbr/dataset/finqa_original/operation_list.txt
Reading: /shared/s3/lab07/yikyung/cbr/dataset/finqa_original/constant_list.txt


In [4]:
train_data, train_examples, op_list, const_list = \
    read_examples(input_path=conf.train_file, case_path=conf.train_case, tokenizer=tokenizer,
                  op_list=op_list, const_list=const_list, log_file=log_file)

Reading /home/ubuntu/yikyung/dataset/finqa_retriever_output/train_retrieve.json
Reading /home/ubuntu/yikyung/dataset/case_retriever_output/bi_encoder/train_retrieved_noise.json


In [6]:
valid_data, valid_examples, op_list, const_list = \
    read_examples(input_path=conf.valid_file, case_path=conf.valid_case, tokenizer=tokenizer,
                  op_list=op_list, const_list=const_list, log_file=log_file)

Reading /shared/s3/lab07/yikyung/cbr/dataset/finqa_retriever_output/dev_retrieve.json
Reading /shared/s3/lab07/yikyung/cbr/dataset/case_retriever_output/cross_encoder/dev_retrieved_noise1_L.json


In [7]:
kwargs = {"examples": valid_examples,
          "tokenizer": tokenizer,
          "max_seq_length": conf.max_seq_length,
          "max_program_length": conf.max_program_length,
          "is_training": True,
          "op_list": op_list,
          "op_list_size": len(op_list),
          "const_list": const_list,
          "const_list_size": len(const_list),
          "verbose": True}

In [8]:
valid_features = convert_examples_to_features(**kwargs)

In [9]:
input_ids = valid_features[0].input_ids
input_mask = valid_features[0].input_mask
segment_ids = valid_features[0].segment_ids
option_mask = valid_features[0].option_mask
program_ids = valid_features[0].program_ids
program_mask = valid_features[0].program_mask
options = valid_features[0].options
answer = valid_features[0].answer
program = valid_features[0].program
program_weight = valid_features[0].program_weight
case_ids = valid_features[0].case_ids
case_mask = valid_features[0].case_mask
case_segs = valid_features[0].case_segs


input_ids = torch.tensor(input_ids).unsqueeze(0)
input_mask = torch.tensor(input_mask).unsqueeze(0)
segment_ids = torch.tensor(segment_ids).unsqueeze(0)
program_ids = torch.tensor(program_ids).unsqueeze(0)
program_mask = torch.tensor(program_mask).unsqueeze(0)
option_mask = torch.tensor(option_mask).unsqueeze(0)
case_ids = torch.tensor(case_ids).unsqueeze(0)
case_mask = torch.tensor(case_mask).unsqueeze(0)
case_segs = torch.tensor(case_segs).unsqueeze(0)


In [10]:
option_mask.size()

torch.Size([1, 1068])

In [11]:
input_mask.size()

torch.Size([1, 512])

In [12]:
from Model_new import Bert_model

In [13]:
model = Bert_model(num_decoder_layers=conf.num_decoder_layers,
                    hidden_size=model_config.hidden_size,
                    dropout_rate=conf.dropout_rate,
                    program_length=conf.max_program_length,
                    input_length=conf.max_seq_length,
                    op_list=op_list,
                    const_list=const_list,
                    tokenizer=tokenizer)
model.eval()
with torch.no_grad():
    logits = model(False, input_ids, input_mask, segment_ids, case_ids, case_mask, case_segs,
                    option_mask, program_ids, program_mask, device=conf.device)


Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


NotImplementedError: Could not run 'aten::mkldnn_rnn_layer' with arguments from the 'CUDA' backend. This could be because the operator doesn't exist for this backend, or was omitted during the selective/custom build process (if using custom build). If you are a Facebook employee using PyTorch on mobile, please visit https://fburl.com/ptmfixes for possible resolutions. 'aten::mkldnn_rnn_layer' is only available for these backends: [CPU, Meta, BackendSelect, Python, FuncTorchDynamicLayerBackMode, Functionalize, Named, Conjugate, Negative, ZeroTensor, ADInplaceOrView, AutogradOther, AutogradCPU, AutogradCUDA, AutogradHIP, AutogradXLA, AutogradMPS, AutogradIPU, AutogradXPU, AutogradHPU, AutogradVE, AutogradLazy, AutogradMeta, AutogradMTIA, AutogradPrivateUse1, AutogradPrivateUse2, AutogradPrivateUse3, AutogradNestedTensor, Tracer, AutocastCPU, AutocastCUDA, FuncTorchBatched, FuncTorchVmapMode, Batched, VmapMode, FuncTorchGradWrapper, PythonTLSSnapshot, FuncTorchDynamicLayerFrontMode, Pytho

In [ ]:
logits.size()

In [12]:
bert_outputs = model.bert(input_ids=input_ids, attention_mask=input_mask, token_type_ids=segment_ids)
bert_sequence_output = bert_outputs.last_hidden_state
bert_pooled_output = bert_sequence_output[:, 0, :]
batch_size, seq_length, bert_dim = list(bert_sequence_output.size())

print(bert_sequence_output.size())

pooled_output = model.cls_prj(bert_pooled_output)
pooled_output = model.cls_dropout(pooled_output)

sequence_output = model.seq_prj(bert_sequence_output)
sequence_output = model.seq_dropout(sequence_output)
print(sequence_output.size())

torch.Size([1, 512, 768])
torch.Size([1, 512, 768])


In [13]:
case_bert_outputs = model.bert(input_ids = case_ids, attention_mask=case_mask, token_type_ids=case_segs)
case_bert_sequence_output = case_bert_outputs.last_hidden_state
case_sequence_output = model.case_seq_prj(case_bert_sequence_output)
case_sequence_output = model.case_seq_dropout(case_bert_sequence_output)

print(case_sequence_output.size())

torch.Size([1, 512, 768])


In [14]:
split_program_ids = torch.split(program_ids, 1, dim=1)
print(model.program_length)
print(program_ids.size())
print(split_program_ids[0].size())

30
torch.Size([1, 30])
torch.Size([1, 1])


In [15]:
# option_size = model.reserved_token_size + seq_length

op_embeddings = model.reserved_token_embedding(model.reserved_ind)
print(op_embeddings.size())
op_embeddings = op_embeddings.repeat(batch_size, 1, 1)
print(op_embeddings.size())

init_decoder_output = model.reserved_token_embedding(model.reserved_go)
print(init_decoder_output.size())
decoder_output = init_decoder_output.repeat(batch_size, 1, 1)
print(decoder_output.size())

if conf.sep_attention:
    decoder_history = decoder_output
else:
    decoder_history = torch.unsqueeze(pooled_output, dim=-1)

torch.Size([44, 768])
torch.Size([1, 44, 768])
torch.Size([1, 768])
torch.Size([1, 1, 768])


In [16]:
initial_option_embeddings = torch.cat(
    [op_embeddings, sequence_output, case_sequence_output], dim=1)
print(initial_option_embeddings.size())

this_step_new_op_emb = initial_option_embeddings

torch.Size([1, 1068, 768])


In [17]:
decoder_state_h = torch.zeros(1, batch_size, model.hidden_size)
decoder_state_c = torch.zeros(1, batch_size, model.hidden_size)
print(decoder_state_c.size())

print(input_mask.size())
float_input_mask = input_mask.float()
float_input_mask = torch.unsqueeze(float_input_mask, dim=-1)
print(float_input_mask.size())


torch.Size([1, 1, 768])
torch.Size([1, 512])
torch.Size([1, 512, 1])


In [18]:
print(case_mask.size())
float_case_mask = case_mask.float()
float_case_mask = torch.unsqueeze(float_case_mask, dim=-1)
print(float_case_mask.size())

torch.Size([1, 512])
torch.Size([1, 512, 1])


In [20]:
decoder_history_attn_vec = model.decoder_history_attn_prj(decoder_output)
decoder_history_attn_vec = model.decoder_history_attn_dropout(decoder_history_attn_vec)
print(decoder_history_attn_vec.size())

decoder_history_attn_w = torch.matmul(decoder_history, torch.transpose(decoder_history_attn_vec, 1, 2))  # (B1H)(BH1)
decoder_history_attn_w = F.softmax(decoder_history_attn_w, dim=1)
print(decoder_history_attn_w.size()) # B11

decoder_history_ctx_embeddings = torch.matmul(torch.transpose(decoder_history_attn_w, 1, 2), decoder_history)
print(decoder_history_ctx_embeddings.size())

torch.Size([1, 1, 768])
torch.Size([1, 1, 1])
torch.Size([1, 1, 768])


In [21]:
question_attn_vec = model.question_attn_prj(decoder_output)
question_attn_vec = model.question_attn_dropout(question_attn_vec)
print(question_attn_vec.size())

question_attn_w = torch.matmul(sequence_output, torch.transpose(question_attn_vec, 1, 2))
question_attn_w -= 1e6 * (1 - float_input_mask)
question_attn_w = F.softmax(question_attn_w, dim=1)
print(question_attn_w.size())

question_ctx_embeddings = torch.matmul(torch.transpose(question_attn_w, 1, 2), sequence_output)
print(question_ctx_embeddings.size())

torch.Size([1, 1, 768])
torch.Size([1, 512, 1])
torch.Size([1, 1, 768])


In [22]:
question_summary_vec = model.question_summary_attn_prj(decoder_output)
question_summary_vec = model.question_summary_attn_dropout(question_summary_vec)
print(question_summary_vec.size())

question_summary_w = torch.matmul(sequence_output, torch.transpose(question_summary_vec, 1, 2))
question_summary_w -= 1e6 * (1 - float_input_mask)
question_summary_w = F.softmax(question_summary_w, dim=1)
print(question_summary_w.size())    # [B 512 H]x[B 768 H]

question_summary_embeddings = torch.matmul(torch.transpose(question_summary_w, 1, 2), sequence_output)
print(question_summary_embeddings.size())

torch.Size([1, 1, 768])
torch.Size([1, 512, 1])
torch.Size([1, 1, 768])


In [23]:
case_attn_vec = model.case_attn_prj(decoder_output)
case_attn_vec = model.case_attn_dropout(case_attn_vec)

case_attn_w = torch.matmul(case_sequence_output, torch.transpose(case_attn_vec, 1, 2)) # attention score
case_attn_w -= 1e6 * (1 - float_input_mask)
case_attn_w = F.softmax(case_attn_w, dim=1) # attention coefficient

case_embeddings = torch.matmul(torch.transpose(case_attn_w, 1, 2), case_sequence_output) # attention value
print(case_embeddings.size())

torch.Size([1, 1, 768])


In [24]:
model.input_embeddings_prj

Linear(in_features=3072, out_features=768, bias=True)

In [25]:
concat_input_embeddings = torch.cat([decoder_history_ctx_embeddings,
                                        question_ctx_embeddings,
                                        case_embeddings,
                                        decoder_output], dim=-1)

print(concat_input_embeddings.size())
input_embeddings = model.input_embeddings_prj(concat_input_embeddings)
input_embeddings = model.input_embeddings_layernorm(input_embeddings)
print(input_embeddings.size())

# torch.Size([1, 1, 2304])
# torch.Size([1, 1, 768])

torch.Size([1, 1, 3072])
torch.Size([1, 1, 768])


In [26]:
question_option_vec = this_step_new_op_emb * question_summary_embeddings # H * att_p'
print(question_option_vec.size())
option_embeddings = torch.cat([this_step_new_op_emb, question_option_vec], dim=-1)  #[H; h*att_p']
print(option_embeddings.size())
option_embeddings = model.option_embeddings_prj(option_embeddings)
print(option_embeddings.size())

option_logits = torch.matmul(option_embeddings, torch.transpose(input_embeddings, 1, 2))
print(option_logits.size())
option_logits = torch.squeeze(option_logits, dim=2)  # [batch, op + seq_len]
option_logits -= 1e6 * (1 - option_mask)
print(option_logits.size())

# torch.Size([1, 556, 1536])
# torch.Size([1, 556, 768])
# torch.Size([1, 556, 1])
# torch.Size([1, 556])

torch.Size([1, 1068, 768])
torch.Size([1, 1068, 1536])
torch.Size([1, 1068, 768])
torch.Size([1, 1068, 1])
torch.Size([1, 1068])


In [31]:
program_index = torch.unsqueeze(
    split_program_ids[0], dim=1)
print(program_index.size())

program_index = torch.repeat_interleave(
    program_index, model.hidden_size, dim=2)  # [batch, 1, hidden]
print(program_index.size())


torch.Size([1, 1, 1])
torch.Size([1, 1, 768])


In [32]:
input_program_embeddings = torch.gather(
    option_embeddings, dim=1, index=program_index)
print(input_program_embeddings.size())

torch.Size([1, 1, 768])


In [34]:
decoder_output, (decoder_state_h, decoder_state_c) = model.rnn(
    input_program_embeddings, (decoder_state_h, decoder_state_c))
print(decoder_output.size())

decoder_history = torch.cat(
    [decoder_history, input_program_embeddings], dim=1)
# (1,1,H) + (1,1,H) = (1,2,H)

torch.Size([1, 1, 768])
